In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, precision_recall_fscore_support

## Adam Experiment

In [3]:
# ==========================================
# 1. PERSIAPAN DATA (CONFIG 2: 60:40)
# ==========================================
print("📂 Loading Data...")
# Pastikan path file benar
df = pd.read_csv('../data/processed_attrition_data.csv') 
X = df.drop(columns=['Attrition_Risk_Level'])
y = df['Attrition_Risk_Level']

# ⚠️ PERUBAHAN DISINI UNTUK CONFIG 2 (60:40)
print("⚙️ Setting Split Data ke 60:40 (Test Size = 0.4)...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y 
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# ==========================================
# 2. VARIABEL EKSPERIMEN (GRID SEARCH)
# ==========================================
list_lr = [0.04, 0.08, 0.06, 0.005, 0.009, 0.007, 0.2, 0.6, 0.012, 0.010, 0.015, 0.018, 0.0111]
list_activation = ['logistic', 'relu', 'tanh']
list_architecture = [(100, 50), (32, 64, 16), (128, 64, 32, 16), (256, 128, 64, 32, 16)]

results = []
counter = 1
total_exp = len(list_lr) * len(list_activation) * len(list_architecture)

print(f"🚀 MEMULAI {total_exp} KOMBINASI EKSPERIMEN (CONFIG 2)...")
print("=" * 60)

# ==========================================
# 3. TRIPLE LOOPING
# ==========================================
for arch in list_architecture:
    for act in list_activation:
        for LR in list_lr:
            
            # Definisi Model
            mlp = MLPClassifier(
                hidden_layer_sizes=arch,
                activation=act,
                solver='adam',
                learning_rate_init=LR,
                momentum=0.8,
                max_iter=375,
                random_state=52,
                early_stopping=True,
                validation_fraction=0.1,
                verbose=False
            )
            
            # Training
            mlp.fit(X_train_scaled, y_train)
            
            # Evaluasi
            y_train_pred = mlp.predict(X_train_scaled)
            mse_train = mean_squared_error(y_train, y_train_pred)
            acc_train = accuracy_score(y_train, y_train_pred)
            
            # Hitung Detail Per Kelas
            precision, recall, f1, support = precision_recall_fscore_support(
                y_train, y_train_pred, labels=[0, 1, 2], zero_division=0
            )
            
            # Simpan Data
            row = {
                "No": counter,
                "Split": "60:40", 
                "Arsitektur": str(arch),
                "Activation": act,
                "Learning Rate": LR,
                "Epoch": mlp.n_iter_,
                "Loss (MSE)": round(mse_train, 5),
                "Acc Train": round(acc_train, 4),
                
                # --- Detail Metrics ---
                "Train Precision 0": round(precision[0], 4),
                "Train Precision 1": round(precision[1], 4),
                "Train Precision 2": round(precision[2], 4),
                
                "Train Recall 0": round(recall[0], 4),
                "Train Recall 1": round(recall[1], 4),
                "Train Recall 2": round(recall[2], 4),
                
                "Train F1 0": round(f1[0], 4),
                "Train F1 1": round(f1[1], 4),
                "Train F1 2": round(f1[2], 4),
                
                "Support 0": support[0],
                "Support 1": support[1],
                "Support 2": support[2]
            }
            results.append(row)
            print(f"✅ Exp {counter}: {act} | LR {LR} | Acc: {acc_train:.2%}")
            counter += 1

# ==========================================
# 4. EXPORT
# ==========================================
df_results = pd.DataFrame(results)

# Nama file dibedakan agar tidak menimpa hasil Config 1
nama_file = "Hasil_GridSearch_Adam_6040.csv" 

df_results.to_csv(nama_file, index=False)
print("\n" + "="*60)
print(f"🎉 SELESAI! File tersimpan sebagai: {nama_file}")
print("Silakan copy isi file ini ke Sheet Config-2 di Spreadsheet.")

📂 Loading Data...
⚙️ Setting Split Data ke 60:40 (Test Size = 0.4)...
🚀 MEMULAI 156 KOMBINASI EKSPERIMEN (CONFIG 2)...
✅ Exp 1: logistic | LR 0.04 | Acc: 98.86%
✅ Exp 2: logistic | LR 0.08 | Acc: 98.96%
✅ Exp 3: logistic | LR 0.06 | Acc: 98.89%
✅ Exp 4: logistic | LR 0.005 | Acc: 99.63%
✅ Exp 5: logistic | LR 0.009 | Acc: 99.32%
✅ Exp 6: logistic | LR 0.007 | Acc: 99.37%
✅ Exp 7: logistic | LR 0.2 | Acc: 96.93%
✅ Exp 8: logistic | LR 0.6 | Acc: 91.34%
✅ Exp 9: logistic | LR 0.012 | Acc: 99.63%
✅ Exp 10: logistic | LR 0.01 | Acc: 99.23%
✅ Exp 11: logistic | LR 0.015 | Acc: 99.32%
✅ Exp 12: logistic | LR 0.018 | Acc: 99.40%
✅ Exp 13: logistic | LR 0.0111 | Acc: 99.07%
✅ Exp 14: relu | LR 0.04 | Acc: 98.84%
✅ Exp 15: relu | LR 0.08 | Acc: 98.79%
✅ Exp 16: relu | LR 0.06 | Acc: 99.20%
✅ Exp 17: relu | LR 0.005 | Acc: 99.41%
✅ Exp 18: relu | LR 0.009 | Acc: 98.76%
✅ Exp 19: relu | LR 0.007 | Acc: 99.63%
✅ Exp 20: relu | LR 0.2 | Acc: 98.19%
✅ Exp 21: relu | LR 0.6 | Acc: 63.96%
✅ Exp 22: re

## SGD Experiment

In [3]:
# ==========================================
# 1. PERSIAPAN DATA (CONFIG 2: 60:40)
# ==========================================
print("📂 Loading Data...")
# Pastikan path file benar
df = pd.read_csv('../data/processed_attrition_data.csv') 
X = df.drop(columns=['Attrition_Risk_Level'])
y = df['Attrition_Risk_Level']

# ⚠️ PERUBAHAN DISINI UNTUK CONFIG 2 (60:40)
print("⚙️ Setting Split Data ke 60:40 (Test Size = 0.4)...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y 
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# ==========================================
# 2. VARIABEL EKSPERIMEN (GRID SEARCH)
# ==========================================
list_lr = [0.04, 0.08, 0.06, 0.005, 0.009, 0.007, 0.2, 0.6, 0.012, 0.010, 0.015, 0.018, 0.0111]
list_activation = ['logistic', 'relu', 'tanh']
list_architecture = [(100, 50), (32, 64, 16), (128, 64, 32, 16), (256, 128, 64, 32, 16)]

results = []
counter = 1
total_exp = len(list_lr) * len(list_activation) * len(list_architecture)

print(f"🚀 MEMULAI {total_exp} KOMBINASI EKSPERIMEN (CONFIG 2)...")
print("=" * 60)

# ==========================================
# 3. TRIPLE LOOPING
# ==========================================
for arch in list_architecture:
    for act in list_activation:
        for LR in list_lr:
            
            # Definisi Model
            mlp = MLPClassifier(
                hidden_layer_sizes=arch,
                activation=act,
                solver='sgd',
                learning_rate_init=LR,
                momentum=0.8,
                max_iter=375,
                random_state=52,
                early_stopping=True,
                validation_fraction=0.1,
                verbose=False
            )
            
            # Training
            mlp.fit(X_train_scaled, y_train)
            
            # Evaluasi
            y_train_pred = mlp.predict(X_train_scaled)
            mse_train = mean_squared_error(y_train, y_train_pred)
            acc_train = accuracy_score(y_train, y_train_pred)
            
            # Hitung Detail Per Kelas
            precision, recall, f1, support = precision_recall_fscore_support(
                y_train, y_train_pred, labels=[0, 1, 2], zero_division=0
            )
            
            # Simpan Data
            row = {
                "No": counter,
                "Split": "60:40", 
                "Arsitektur": str(arch),
                "Activation": act,
                "Learning Rate": LR,
                "Epoch": mlp.n_iter_,
                "Loss (MSE)": round(mse_train, 5),
                "Acc Train": round(acc_train, 4),
                
                # --- Detail Metrics ---
                "Train Precision 0": round(precision[0], 4),
                "Train Precision 1": round(precision[1], 4),
                "Train Precision 2": round(precision[2], 4),
                
                "Train Recall 0": round(recall[0], 4),
                "Train Recall 1": round(recall[1], 4),
                "Train Recall 2": round(recall[2], 4),
                
                "Train F1 0": round(f1[0], 4),
                "Train F1 1": round(f1[1], 4),
                "Train F1 2": round(f1[2], 4),
                
                "Support 0": support[0],
                "Support 1": support[1],
                "Support 2": support[2]
            }
            results.append(row)
            print(f"✅ Exp {counter}: {act} | LR {LR} | Acc: {acc_train:.2%}")
            counter += 1

# ==========================================
# 4. EXPORT
# ==========================================
df_results = pd.DataFrame(results)

# Nama file dibedakan agar tidak menimpa hasil Config 1
nama_file = "Hasil_GridSearch_SGD_6040.csv" 

df_results.to_csv(nama_file, index=False)
print("\n" + "="*60)
print(f"🎉 SELESAI! File tersimpan sebagai: {nama_file}")
print("Silakan copy isi file ini ke Sheet Config-2 di Spreadsheet.")

📂 Loading Data...
⚙️ Setting Split Data ke 60:40 (Test Size = 0.4)...
🚀 MEMULAI 156 KOMBINASI EKSPERIMEN (CONFIG 2)...
✅ Exp 1: logistic | LR 0.04 | Acc: 99.68%
✅ Exp 2: logistic | LR 0.08 | Acc: 99.26%
✅ Exp 3: logistic | LR 0.06 | Acc: 99.58%
✅ Exp 4: logistic | LR 0.005 | Acc: 50.00%
✅ Exp 5: logistic | LR 0.009 | Acc: 50.00%
✅ Exp 6: logistic | LR 0.007 | Acc: 50.00%
✅ Exp 7: logistic | LR 0.2 | Acc: 99.33%
✅ Exp 8: logistic | LR 0.6 | Acc: 98.99%
✅ Exp 9: logistic | LR 0.012 | Acc: 99.04%
✅ Exp 10: logistic | LR 0.01 | Acc: 50.00%
✅ Exp 11: logistic | LR 0.015 | Acc: 99.11%
✅ Exp 12: logistic | LR 0.018 | Acc: 99.24%
✅ Exp 13: logistic | LR 0.0111 | Acc: 98.79%
✅ Exp 14: relu | LR 0.04 | Acc: 99.68%
✅ Exp 15: relu | LR 0.08 | Acc: 98.88%
✅ Exp 16: relu | LR 0.06 | Acc: 99.29%
✅ Exp 17: relu | LR 0.005 | Acc: 99.00%
✅ Exp 18: relu | LR 0.009 | Acc: 99.51%
✅ Exp 19: relu | LR 0.007 | Acc: 99.20%
✅ Exp 20: relu | LR 0.2 | Acc: 99.28%
✅ Exp 21: relu | LR 0.6 | Acc: 98.88%
✅ Exp 22: re